# Preparing our fraudulent regression example
Here we'll import the fraud picture, transform it
to a binary matrix and then create our regression
data set.  The calculations here are derived
from Stefanski's American Statistician Paper
(https://www4.stat.ncsu.edu/~stefanski/NSF_Supported/Hidden_Images/Residual_Surrealism_TAS_2007.pdf)


In [ ]:
from PIL import Image
import numpy as np
import numpy.random as nr
from scipy.linalg import sqrtm, pinv
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import scale
import pandas as pd
from pandas.plotting import scatter_matrix

# Read image as grayscale and downscale by factor of seven
fraud_img = np.array(
    Image.open("unc_smells.jpg").convert('L')
)[0::10, 0::10]
pixel_threshold = 100
binary_fraud_img = np.array(fraud_img < pixel_threshold)

# Compute y_hat, residual pairs
n = binary_fraud_img.shape[0]
nonzero_ind = [(i,j) for i in range(n) for j in range(n) if binary_fraud_img[n-i-1,j]]
fitted_values = np.array([x[1] for x in nonzero_ind])
residuals = np.array([x[0] for x in nonzero_ind])
plt.scatter(fitted_values, residuals)
plt.show()

In [2]:
## Compute X, Y so that regression coeffs are target beta
## and corr among X are AR(\rho)
sample_size = len(residuals)
Y = fitted_values + residuals
res_projection = np.identity(sample_size) - \
                 np.outer(residuals, residuals)/np.dot(residuals, residuals)
fit_projection = np.identity(sample_size) - \
                 np.outer(fitted_values, fitted_values)/np.dot(fitted_values, fitted_values)

p = 16 # Dimension of linear model
M = nr.randn(sample_size*p).reshape((sample_size, p))
X = scale(np.hstack((
    fitted_values.reshape((sample_size, 1)),
    np.dot(np.dot(res_projection, fit_projection), M)
)))

In [3]:
target_beta = np.ones(p+1)*0.5
target_beta[0:5] = np.array([3.24, 3, 5, -4, 1])
rho = 0.5
target_correlation = np.identity(p+1)
for i in range(p+1):
    for j in range(i):
        target_correlation[i,j] = rho**np.abs(i-j)
        target_correlation[j,i] = target_correlation[i,j]
target_correlation_sqrt = sqrtm(target_correlation)

In [4]:
Q1 = (nr.randn((p+1)**2)).reshape((p+1,p+1))
X1 = np.dot(X, Q1)
Q2 = np.sqrt(sample_size)*np.dot(
    pinv(sqrtm(np.dot(np.transpose(X1), X1))),
    target_correlation_sqrt
    )
X2 = np.dot(X1, Q2)
X2_lm_fit = linear_model.LinearRegression().fit(X2, Y)
X2_beta_hat = X2_lm_fit.coef_
Q3 = np.dot(np.diag(X2_beta_hat), np.diag(1/target_beta))

In [5]:
# Check results
lm_fit = linear_model.LinearRegression().fit(X_final, Y)
lm_fitted = lm_fit.predict(X_final)
lm_residuals = Y - lm_fitted
plt.scatter(lm_fitted, lm_residuals, s=1)
plt.show()

NameError: name 'X_final' is not defined

In [ ]:
df = pd.DataFrame(np.hstack((
    Y.reshape(sample_size, 1), X_final
)))
scatter_matrix(df, s=0.5, figsize=(10,10))
plt.show()

## Give columns names to make it more realistic
df.columns = ["survival_normalized",
              *["g" + str(i) for i in range(p+1)]]
df.to_csv("lineberger_study_data.csv", sep=",")